<a href="https://colab.research.google.com/github/NicoAnsaldi/Metodos/blob/master/Copy_of_taller_1_mn2019_1c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1 - Métodos Numéricos
## 2019 - 1er cuatrimestre
### Objetivo y evaluación

El objetivo del trabajo es completar los siguientes puntos relacionados con los temas de factorización LU, de Cholesky y matrices ortogonales.

#### Evaluación:
- Coloquio con los docentes durante la clase (preguntas sobre el enunciado y el código). Justificar las respuestas
- En caso de no asistir a clase o no terminar a tiempo, se debe entregar la resolución del taller con las respuestas justificadas por *escrito* hasta el viernes 19 de abril a [metnum.lab@gmail.com](metnum.lab)
- Implementar cada uno de los siguientes ejercicios, haciendo que corran los respectivos tests.


In [1]:
#Código para que corran los tests: (pueden ignorarlo, pero ejecútenlo al principio de todo)


import numpy as np
import math
import traceback


def mntest(func):
    global tests
    
    tests.append(func)    
    
    return func

def correr_tests():
    excepciones = []
    for test in tests:
        try:
            print("Corriendo {} ... ".format(test.__name__), end='')
            test()
            print("OK")
        except AssertionError as e:
            error_msg = traceback.format_exc()
            excepciones.append((test, error_msg))
            print("ERROR")
    
    if len(excepciones) > 0:
        print("\nErrores:\n")
        for (test, error_msg) in excepciones:
            print("En {}".format(test.__name__))
            print(error_msg)
    else:
        print("Todos los tests pasaron correctamente")



## Ejercicio 1

Repasar la definición de factorización LU. ¿Bajo qué condiciones podemos garantizar
que una matriz inversible tiene factorización LU? 

Completar la función `tiene_lu` que dada una matriz A, nos devuelve `True` si tiene factorización LU, o `False` en caso contrario.

**Tip**: considerar las funciones `shape` para obtener las dimensiones de una matriz, y `np.linalg.det` para obtener el determinante de una matriz.


In [32]:
eps = 1e-6

def tiene_lu(A):
    N = A.shape[0]
    for j in range(0, N):
        if (np.linalg.det(A[0:j, 0:j]) == 0):
            return False
    return True
    
    """
    Dada una matriz A, devuelve True si tiene descomposición LU, o False en caso contrario
    
    Argumentos:
    -----------
    
    A: np.array
        Matriz de floats

    Devuelve:
    ---------
    
    res: bool
        True si tiene LU, False caso contrario
    """
    pass
    

In [33]:
tests = []


@mntest
def testear_identidad_tiene_LU():
    A = np.identity(3)
        
    assert(tiene_lu(A))
    

@mntest
def testear_matriz_ceros_no_tiene_LU():
    A = np.zeros((3, 3))
        
    assert(not tiene_lu(A))
    
@mntest
def testear_matriz_no_inversible():
    A = np.ones((3, 3))
    
    assert(not tiene_lu(A))

@mntest 
def testear_matriz_permutacion_no_tiene_LU():
    A = np.array([
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])
    
    assert(not tiene_lu(A))


correr_tests()

Corriendo testear_identidad_tiene_LU ... OK
Corriendo testear_matriz_ceros_no_tiene_LU ... OK
Corriendo testear_matriz_no_inversible ... OK
Corriendo testear_matriz_permutacion_no_tiene_LU ... OK
Todos los tests pasaron correctamente


## Ejercicio 2

Calcular LU en bloques para que tome una matrix $A \in \mathbb{R}^{n \times n}$ y nos devuelva $L$ y $U$, su descomposición correspondiente

**Hint**: ojo con la dimensión de los vectores. Si queremos multiplicar una matriz "columna", con una matriz "fila" tenemos que hacer reshape primero.

In [34]:
def lu_en_bloques(A):
    """
    Dada una matriz A, devuelve L, U 
    
    Argumentos:
    -----------
    
    A: np.array
        Matriz de floats

    Devuelve:
    ---------
    
    L, U: np.array
        Descomposición LU de A
        

    """    
    if not tiene_lu(A):
        raise ValueError("La Matriz no tiene descomposición LU")
    
    current_matrix = np.matrix(A)
    L = np.zeros(A.shape)
    U = np.zeros(A.shape)
    
    n = A.shape[0]
  
    for i in range(0,n):
      L.itemset(i,i,1.)
      U[i,i:] = current_matrix[i,i:]
      L[i+1:,i] = np.reshape (current_matrix[i+1:,i]/U[i,i], (-1)) 
      vectorColumnaL = np.reshape(L[i+1:,i], (-1,1))
      vectorFilaU = np.reshape(U[i,i+1:], (1,-1))
      aux = vectorColumnaL @ vectorFilaU
      current_matrix[i+1:, i+1:] = current_matrix[i+1: , i+1:] - aux
      pass
    return L, U    

In [35]:
tests = []


@mntest
def testear_con_multiplo_identidad():
    A = 3 * np.identity(3)
    
    L, U = lu_en_bloques(A)
    
    assert(np.allclose(L, np.eye(3)))
    assert(np.allclose(U, 3*np.eye(3)))
    
    

@mntest
def testear_con_otra_matriz():
    L = np.array([
        [1, 0, 0],
        [1, 1, 0],
        [1, 1, 1],
    ],float)

    U = np.array([
        [1, 1, 1],
        [0, 2, 2],
        [0, 0, 3],
    ],float)

    A = L @ U
   
    
    L1, U1 = lu_en_bloques(A)
    assert(np.allclose(L1, L))
    assert(np.allclose(U1, U))

@mntest
def testear_con_otra_matriz2():
    A = np.array([
        [8, 2, 0],
        [4, 9, 4],
        [6, 7, 9],
    ],float)


    
    L1, U1 = lu_en_bloques(A)
    assert(np.allclose(L1@U1, A))

    

correr_tests()


Corriendo testear_con_multiplo_identidad ... OK
Corriendo testear_con_otra_matriz ... OK
Corriendo testear_con_otra_matriz2 ... OK
Todos los tests pasaron correctamente


## Ejercicio 3

Enunciar un método basado en vectores aleatorios para mostrar que una matriz no es definida positiva o para ganar más confianza en que sí lo es. Completar la función `tiene_sdp_vectores_aleatorios`

Tratar de realizar una implementación sin for, ni while ¿Puede pensar algún ejemplo donde fallaría este método?

**Tip**: Usar las funciones `np.random.randn` con los tamaños. Las funciones `np.array.diagonal, np.array.any` pueden ser de ayuda.

In [36]:
def es_simetrica(A):
    return np.allclose(A, A.T, eps)
       

def tiene_sdp_vectores_aleatorios(A, n):
    """
    Chequea que la matriz sea SDP usando método probabilístico de vectores aleatorios
    
    Argumentos:
    ----------
    
    A: np.array
        Matriz a verificar su condición de SDP
       
    n: int
        Cantidad de vectores a 
    
    Devuelve:
    ---------
    
    res: bool
        True si no encontró si A es SDP bajo este método probabilístico
    """
    if not es_simetrica(A):
        return False
    
    array = np.random.randn(n,A.shape[0])
    
    for i in range(0,n):
      x = array[i]
      xt = array[i]
      
      x=np.reshape(x, (A.shape[0],1))
      xt=np.reshape(xt, (1,A.shape[0]))
      
      if((xt@(A@x)) <= 0):
        return False
    
    return True
    
   
    """
    Completar código acá
    """ 

In [37]:
tests = []


@mntest
def testear_con_identidad():
    A = np.eye(3)
    assert(tiene_sdp_vectores_aleatorios(A, 10000))
    
    

@mntest
def testear_con_matriz_no_sdp():
    A = np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, 1],
    ])
    
    assert(not tiene_sdp_vectores_aleatorios(A, 10000))
    
    
@mntest
def testear_con_otra_matriz_no_sdp():
    A = np.array([
        [1, 0, 0],
        [0, 1, 2],
        [0, 2, 1],
    ])
    
    assert(not tiene_sdp_vectores_aleatorios(A, 10000))
    
    

correr_tests()


Corriendo testear_con_identidad ... OK
Corriendo testear_con_matriz_no_sdp ... OK
Corriendo testear_con_otra_matriz_no_sdp ... OK
Todos los tests pasaron correctamente


## Ejercicio número 4

Completar la función `chol_from_lu` para que tome una matriz A y devuelva la matriz correspondiente a su factorización de Cholesky. En este caso se comienza teniendo la factorización LU de A y se quiere generar la factorización de Cholesky.

En caso de que la matriz no tenga factorización de Cholesky, se debe tirar una excepción `ValueError`
    
**Hint**: considerar la función `np.diag` y `np.sqrt`


In [38]:
def chol_from_lu(A):
    """
    Devuelve la L de Cholesky a través de la factorización LU de la matriz A
    
    En caso de que no tenga LU o que no tenga Cholesky, lanza ValueError
    
    Argumentos:
    ----------
    
    A: np.array
        Matriz a factorizar
    
    Devuelve:
    ---------
    
    L: np.array
        Factorización de Cholesky de A
    """
    if (not es_simetrica(A)):
        raise ValueError("Matriz no simétrica")
    
    """
    Completar acá
    """
    
    L, U = lu_en_bloques(A)
    D = np.diag(U)
    n = A.shape[0]
    for i in range(0,n):
        for j in range(0,i+1):
            if(D.item(j)<0):
                raise ValueError("No existe cholesky")
            L.itemset(i,j, L.item(i,j)*np.sqrt(D.item(j)))
    return L
   

In [39]:
tests = []


@mntest
def testear_con_identidad():
    A = np.eye(3)
    L = chol_from_lu(A)
    
    # Cholesky es la identidad también :-)
    assert(np.allclose(A, L))
    

@mntest
def testear_con_multiplo_identidad():
    A = 4 * np.eye(3)
    L = chol_from_lu(A)
    
    # Cholesky es la identidad también :-)
    assert(np.allclose(L, 2* np.eye(3)))
    
@mntest
def testear_con_matriz_no_sdp():
    A = np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, 2]
    ])
    try:
        L = chol_from_lu(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass

@mntest
def testear_con_matriz():
    L1 = np.array([
        [1, 0, 0],
        [2, 2, 0],
        [4, 4, 4]
    ])
    
    A = L1 @ L1.T
    L = chol_from_lu(A)
    assert(np.allclose(L, L1))

correr_tests()


Corriendo testear_con_identidad ... OK
Corriendo testear_con_multiplo_identidad ... OK
Corriendo testear_con_matriz_no_sdp ... OK
Corriendo testear_con_matriz ... OK
Todos los tests pasaron correctamente


## Ejercicio 5

Completar la función `chol_en_bloques` para que tome una matriz A y devuelva la matriz L correspondiente a su factorización de Cholesky.
    

In [40]:

def chol_en_bloques(A):
    """
    Devuelve la L de Cholesky a través del algoritmo iterativo por bloques
    
    En caso de que no tenga LU o que no tenga Cholesky, lanza ValueError
    
    Argumentos:
    ----------
    
    A: np.array
        Matriz a factorizar
    
    Devuelve:
    ---------
    
    L: np.array
        Factorización de Cholesky de A
    """
    if not es_simetrica(A):
        raise ValueError("Matriz no simétrica")

    n = A.shape[0]
    current_matrix = np.matrix(A)
    # Inicializo L con ceros
    L = np.zeros((n,n))
    # Calculo de sub-bloques de L.
    
    for i in range(0,n):
        if(current_matrix[i,i] < 0):
            raise ValueError("No tiene Cholesky")
        
        L.itemset(i,i,np.sqrt(current_matrix[i,i]))
        L[i+1: ,i] = np.reshape(current_matrix[i+1: ,i]/L[i,i], (-1))
        vectorColumnaL = np.reshape(L[i+1:,i], (-1,1))
        vectorFilaL = np.reshape(L[i+1:,i], (1,-1))
        aux = vectorColumnaL @ vectorFilaL
        current_matrix[i+1:, i+1:] = current_matrix[i+1: , i+1:] - aux
        # Calculo todos los subbloques de L
        # COMPLETAR
        pass
    return L

In [41]:

tests = []

@mntest
def matriz_diagonal_positiva():
    A = np.array([
          [4.0,.0],
          [.0,9.0]
    ])
    L = np.array([
          [2.0,0.0],
          [.0,3.0]
    ])
    res = chol_en_bloques(A)
    assert(np.allclose(res,L))
    
    

@mntest
def matriz_no_simetrica():
    A = np.array([
          [0.066076,.181880],
          [1.181880,.624953]
    ])

    try:
        L = chol_en_bloques(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass

@mntest
def matriz_no_sdp():
    A = np.array([
          [-0.066076,   0.181880],
          [ 0.181880,   0.624953]
    ])

    try:
        L = chol_en_bloques(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass

@mntest
def matriz_copada():
       
    L_1 =  np.array([
        [5.99246,   .0,  .0,   .0,   .0],
        [0.45174,   5.02065,   .0,   .0,   .0],
        [0.50921,   0.40611,   5.25767,   .0,   .0],
        [0.45632,   0.52172,   0.78430,   5.98219,   .0],
        [0.41578,   0.11483,   0.24938,   0.45188,   5.28701]
    ])

    A = L_1@L_1.T
    res = chol_en_bloques(A)
    assert(np.allclose(res,L_1))

   

  
correr_tests()

Corriendo matriz_diagonal_positiva ... OK
Corriendo matriz_no_simetrica ... OK
Corriendo matriz_no_sdp ... OK
Corriendo matriz_copada ... OK
Todos los tests pasaron correctamente


## Ejercicio 6

Completar la función `es_ortogonal`para determinar si una matriz es ortogonal o no. 

En caso de serlo, explicar cómo se podría devolver un conjunto ortonormal de vectores a partir de la matriz.

A su vez, completar los tests con matrices de Givens y Householder. Testear que son ortogonales

In [42]:
def es_ortogonal(A):
    """
    Devuelve True si A es ortogonal, False en otro caso
    
    
    Argumentos:
    ----------
    
    A: np.array
        
    Devuelve:
    ---------
    
    res: bool
        True si A ortogonal, False en otro caso
    """
    I = np.identity(A.shape[0])
    
    return np.allclose(I,A@A.T)
    
    pass

In [43]:
from math import cos, sin
tests = []

@mntest
def probar_con_identidad():
    A = np.eye(3)
    
    assert(es_ortogonal(A))
    
@mntest
def probar_con_givens():
    angle = math.pi/4
    
    
    A = np.array([[math.cos(angle),math.sin(angle)*-1] ,[math.cos(angle) ,math.sin(angle)]])
    
    assert(es_ortogonal(A))

@mntest
def probar_con_householder():
    v = np.array([[1/math.sqrt(3.),1/math.sqrt(3.),1/math.sqrt(3.)]])
    vt = np.reshape(v,(1,3))
    v = np.reshape(v,(3,1))
    A = np.identity(v.shape[0]) - 2*(v@vt)
    assert(es_ortogonal(A))

    
correr_tests()

Corriendo probar_con_identidad ... OK
Corriendo probar_con_givens ... OK
Corriendo probar_con_householder ... OK
Todos los tests pasaron correctamente
